In [1]:
import prov_ref_node_creator as prnc
import networkx as nx
import pandas as pd
import pdb
import os
import two_imgs_eff
import pickle
from get_all_feature_vecs17 import info_storer, info_storer_all
import matplotlib.pyplot as plt
# import disp_img
import imshow_coll
import numpy as np
from itertools import chain

In [28]:
def contracted_nodes(G, u, v, self_loops=True):
    """Returns the graph that results from contracting `u` and `v`.
    Node contraction identifies the two nodes as a single node incident to any
    edge that was incident to the original two nodes.
    Parameters
    ----------
    G : NetworkX graph
       The graph whose nodes will be contracted.
    u, v : nodes
       Must be nodes in `G`.
    self_loops : Boolean
       If this is True, any edges joining `u` and `v` in `G` become
       self-loops on the new node in the returned graph.
    Returns
    -------
    Networkx graph
       A new graph object of the same type as `G` (leaving `G` unmodified)
       with `u` and `v` identified in a single node. The right node `v`
       will be merged into the node `u`, so only `u` will appear in the
       returned graph.
    Examples
    --------
    Contracting two nonadjacent nodes of the cycle graph on four nodes `C_4`
    yields the path graph (ignoring parallel edges)::
        >>> import networkx as nx
        >>> G = nx.cycle_graph(4)
        >>> M = nx.contracted_nodes(G, 1, 3)
        >>> P3 = nx.path_graph(3)
        >>> nx.is_isomorphic(M, P3)
        True
    See also
    --------
    contracted_edge
    quotient_graph
    Notes
    -----
    This function is also available as `identified_nodes`.
    """

    H = G.copy()
    if H.is_directed():
        in_edges = ((w, u, d) for w, x, d in G.in_edges(v, data=True)
                    if self_loops or w != u)
        out_edges = ((u, w, d) for x, w, d in G.out_edges(v, data=True)
                     if self_loops or w != u)
        new_edges = chain(in_edges, out_edges)
    else:
#         new_edges = ((u, w, d) for x, w, d in G.edges(v, data=True)
#                      if self_loops or w != u)
        new_edges = ((u, w, d) for x, w, d in G.edges(v, data=True)
                     if self_loops or w != u)
        new_edges = list()
        nodes_u = [w for x,w in G.edges(u)]
        for x,w,d in G.edges(v, data=True):
            if w != u:
                if w not in nodes_u:
                    new_edges.append((u, w, d))
                else:
                    max_d = max(d, G[u][w])
                    new_edges.append((u,w,max_d))
                    
    v_data = H.node[v]
#     pdb.set_trace()
    H.remove_node(v)
    H.add_edges_from(new_edges)
    if 'contraction' in H.node[u]:
        H.node[u]['contraction'][v] = v_data
    else:
        H.node[u]['contraction'] = {v: v_data}
    return H

In [27]:
%matplotlib
G = nx.Graph()
G.add_edge(1,3,weight=0.5)
G.add_edge(1,5,weight=0.5)
G.add_edge(1,7,weight=0.5)
G.add_edge(0,2,weight=0.6)
G.add_edge(0,4,weight=0.6)
G.add_edge(0,6,weight=0.6)
G.add_edge(1,6,weight=0.7)
# nx.draw(G)
G.edges(data=True)

Using matplotlib backend: TkAgg


[(0, 2, {'weight': 0.6}),
 (0, 4, {'weight': 0.6}),
 (0, 6, {'weight': 0.6}),
 (1, 3, {'weight': 0.5}),
 (1, 5, {'weight': 0.5}),
 (1, 6, {'weight': 0.7}),
 (1, 7, {'weight': 0.5})]

In [31]:
H = contracted_nodes(G,1,0)
H.edges(data=True)

[(1, 2, {'weight': 0.6}),
 (1, 3, {'weight': 0.5}),
 (1, 4, {'weight': 0.6}),
 (1, 5, {'weight': 0.5}),
 (1, 6, {'weight': 0.7}),
 (1, 7, {'weight': 0.5})]

In [36]:
M = nx.contracted_nodes(G,1,0)
M.edges(data=True)

[(1, 2, {'weight': 0.6}),
 (1, 3, {'weight': 0.5}),
 (1, 4, {'weight': 0.6}),
 (1, 5, {'weight': 0.5}),
 (1, 6, {'weight': 0.6}),
 (1, 7, {'weight': 0.5})]

In [35]:
H = contracted_nodes(G,0,1)
# H.edges(data=True)
# G.node[1]
self_loops = False
u = 1
v = 0
new_edges = ((u, w, d) for x, w, d in G.edges(v, data=True)
                     if self_loops or w != u)
new_edges = list()
nodes_u = [w for x,w in G.edges(u)]
for x,w,d in G.edges(v, data=True):
    if w != u:
        if w not in nodes_u:
            new_edges.append((u, w, d))
        else:
            max_d = max(d, G[u][w])
            new_edges.append((u,w,max_d))
dir(new_edges)
for i in new_edges:
    print i

(1, 2, {'weight': 0.6})
(1, 4, {'weight': 0.6})
(1, 6, {'weight': 0.7})


In [55]:
class complex:
    def __init__(self, r, i):
        self.r = r
        self.i = i
G1 = nx.Graph()
c1 = complex(3,4)
G1.add_edge(c1, complex(4,5), weight=0.1)
# G1.edges(data=True)[0][0].r
a1 = G1[G1.nodes()[0]]
k1 = a1.keys()[0]
a1[k1]['weight']

0.1